In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json

import pandas as pd
from transformers import AutoTokenizer

from rate_severity_of_toxic_comments.utilities import *
from rate_severity_of_toxic_comments.model import *
from rate_severity_of_toxic_comments.dataset import *
from rate_severity_of_toxic_comments.training import * 

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
default = open("config/default.json")
CONFIG = json.load(default)

with open("config/local.json") as local:
    CONFIG.update(json.load(local))

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])
# CONFIG['group'] = f'{HASH_NAME}-Baseline'

In [ ]:
df = pd.read_csv("res/data/weighted_train_small.csv")
# df = pd.read_csv("res/data/validation_data.csv")
df = df.sample(20)
data_size = len(df.index)

In [ ]:
train_split = 0.7
threshold_index = int(train_split * data_size)

df_train = df[:threshold_index].reset_index(drop=True)
df_valid = df[threshold_index:].reset_index(drop=True)
training_data, val_data = build_datasets([df_train, df_valid], CONFIG, ["weighted", "weighted"])

In [ ]:
stats = run_training(training_data, val_data, log_interval=10, config=CONFIG, verbose=True)